In [1]:
from astropy.io import fits
import clevar
import pandas as pd
from astropy.table import Table
import astropy
from matplotlib import pyplot as plt
import numpy as np
from numcosmo_py import Ncm, Nc
from numcosmo_py.external.pyssc import pyssc as PySSC
Ncm.cfg_init()

In [2]:
hdul_halos = fits.open("JPAS_mergedproc2a.fits")
hdul_ame = fits.open("richness_mcut_SNR 12_ame.fits")

table_halos = hdul_halos[1].data
table_ame = hdul_ame[1].data

table_halos = table_halos[(table_halos['upid'] == -1)  &  (table_halos['Mvir'] >= 10**12) & (table_halos['Mvir'] <= 10**15)]

print(table_halos.size)
print(table_ame.size)

#print(table_halos.columns)

1212558
23751


In [3]:
catalog_halos = clevar.ClCatalog('Halo catalog',ra=table_halos['RA'] ,dec=table_halos['DEC'] 
                                ,z=table_halos['redshift_R'], mass=table_halos['Mvir'])

catalog_ame = clevar.ClCatalog('Detection catalog', ra=table_ame['RA'], dec=table_ame['DEC']
                             , z=table_ame['z'], mass=table_ame['R_ame']  , mass_err=table_ame['R_ame_err'] ,z_err=table_ame['z_err'])

/global/u2/l/lettieri/miniforge3/envs/numcosmo_developer/lib/python3.12/site-packages/clevar-0.15.3-py3.12.egg/clevar/catalog/tagdata.py:278: UserWarning: id column missing, additional one is being created.
  warnings.warn(f"{id_name} column missing, additional one is being created.")


In [ ]:
mt = clevar.match.ProximityMatch()

match_config = {
    'type': 'cross', # options are cross, cat1, cat2
    'which_radius': 'max', # Case of radius to be used, can be: cat1, cat2, min, max
    'preference': 'angular_proximity', # options are more_massive, angular_proximity or redshift_proximity
    'catalog1': {'delta_z':.2,
                'match_radius': '1 mpc'
                },
    'catalog2': {'delta_z':.2,
                'match_radius': '100 arcsec'
                }
}

cosmo = clevar.cosmology.AstroPyCosmology()

mt.match_from_config(catalog_halos, catalog_ame, match_config, cosmo=cosmo)
mt.save_matches(catalog_halos, catalog_ame, out_dir='temp', overwrite=True)


## ClCatalog 1
## Prep mt_cols
### Prep z_cols
* zmin|zmax from config value
### Prep ang_cols
* ang radius from set scale

## ClCatalog 2
## Prep mt_cols
### Prep z_cols
* zmin|zmax from config value
### Prep ang_cols
* ang radius from set scale

## Multiple match (catalog 1)
Finding candidates (Halo catalog)
* 150,268/1,212,558 objects matched.s

## Multiple match (catalog 2)
Finding candidates (Detection catalog)
* 23,694/23,751 objects matched.

## Finding unique matches of catalog 1
Unique Matches (Halo catalog)


In [ ]:
#display(catalog_halos)
#display(catalog_ame)
mt = clevar.match.ProximityMatch()
mt.load_matches(catalog_halos, catalog_ame, out_dir='temp')

mt1, mt2 = clevar.match.get_matched_pairs(catalog_halos, catalog_ame, 'cross')



zbins = np.linspace(0, 0.664, 15)
mbins = np.logspace(13, 15, 8)
info = clevar.match_metrics.recovery.plot(mt1, 'cross', zbins, mbins, shape='steps')
plt.show()
info = clevar.match_metrics.recovery.plot(mt2, 'cross', zbins, mbins, shape='line')
plt.show()

from clevar.match_metrics.scaling import ClCatalogFuncs as s_cf

s_cf.plot(mt1, mt2, 'cross', col='mass', xscale='log', yscale='log')
plt.show()
s_cf.plot(mt1, mt2, 'cross', col='z')
plt.show()

In [ ]:
area = 4109.3
cosmo = Nc.HICosmoDEXcdm()
cosmo.param_set_by_name("H0", 67.74)
cosmo.param_set_by_name("Omegab", 0.0486)
cosmo.param_set_by_name("Omegax", 0.6911)
cosmo.param_set_by_name("Omegac", 0.2603)
cosmo.param_set_by_name("w", -1.0)


prim = Nc.HIPrimPowerLaw.new()
prim.param_set_by_name("ln10e10ASA", 3.02745)
prim.param_set_by_name("n_SA", 0.9667)

reion = Nc.HIReionCamb.new()

cosmo.add_submodel(prim)
cosmo.add_submodel(reion)

dist = Nc.Distance.new(2.0)

tf = Nc.TransferFuncEH()

psml = Nc.PowspecMLTransfer.new(tf)
psml.require_kmin(1.0e-6)
psml.require_kmax(1.0e3)

psf = Ncm.PowspecFilter.new(psml, Ncm.PowspecFilterType.TOPHAT)
psf.set_best_lnr0()

mulf = Nc.MultiplicityFuncTinker.new()
mulf.set_mdef(Nc.MultiplicityFuncMassDef.CRITICAL)
mulf.set_Delta(200.0)

hmf = Nc.HaloMassFunction.new(dist, psf, mulf)
hmf.set_area(area * (np.pi / 180) ** 2)
hmf.prepare(cosmo)
hbias_Tinker = Nc.HaloBiasTinker.new(hmf)
cad = Nc.ClusterAbundance.new(hmf, None)
cad.set_area(area * (np.pi / 180) ** 2)
hmf.prepare(cosmo)

#σ_8 = 0.8159

In [ ]:
#mass proxy
cluster_m = Nc.ClusterMassAscaso(lnRichness_min = 0.0 ,lnRichness_max = np.log(60.0))
cluster_m.param_set_by_name("mup0", 4.9)
cluster_m.param_set_by_name("mup1", 0.993)
cluster_m.param_set_by_name("mup2", 0)
cluster_m.param_set_by_name("sigmap0", 0.26)
cluster_m.param_set_by_name("sigmap1", 0.0)
cluster_m.param_set_by_name("sigmap2", 0.0)

cluster_m.set_property("mup0_fit", True)
cluster_m.set_property("mup1_fit", True)
cluster_m.set_property("sigmap0_fit", True)

#photo-z
cluster_z = Nc.ClusterPhotozGauss (pz_min = 0.0 , pz_max = 0.7)

print("\n")
print(cluster_m.p(cosmo , 35.45, 0.1 , [35.46546] , [1.175]))
print(cluster_m.intp(cosmo , 36 , 0.1))
print(cluster_m.intp_bin(cosmo , 30.45 , 0.1 , [1] , [100] , [2.90969]))
print("\n")

print(cluster_z.p(cosmo , 32 , 0.66 , [0.657513] , [0,0.007513]))
print(cluster_z.intp(cosmo , 32 , 0.9))
print(cluster_z.intp_bin(cosmo , 32 , 0.9, [0.8] , [1.0] , [0,0.0513]))
print("\n")

In [ ]:
masses  = np.linspace(np.log(10) *12.2 , np.log(10) * 14.5,10000)
masses_10  = np.linspace(12.2 ,14.5,10000)
relation = []

for i in range(len(masses)):
    relation.append(cluster_m.p(cosmo , masses[i], 0.1 , [9.52] , [1.826111]))
plt.plot(masses_10 , relation)
plt.ylabel(r'$P(ln\lambda|M,z)$')
plt.xlabel(r'$lnM$')
plt.show()

masses  = np.linspace(np.log(10) *14.0, np.log(10) * 16  ,10000)
masses_10  = np.linspace(14.0 ,16    ,10000)
relation = []

for i in range(len(masses)):
    relation.append(cluster_m.p(cosmo , masses[i], 0.1 , [338.48] , [1.942005]))
plt.plot(masses_10 , relation)
plt.ylabel(r'$P(ln\lambda|M,z)$')
plt.xlabel(r'$lnM$')
plt.show()

In [ ]:

mset = Ncm.MSet.new_array([cosmo, cluster_m, cluster_z])
ncdata = Nc.DataClusterNCount.new(cad, "NcClusterPhotozGauss", "NcClusterMassRichness")
ncdata.set_init(True)

bin_test =  detections_catalog.query(' R_ame >= 0  & R_ame <= 100')
display(bin_test["R_ame"].sort_values())

richness = np.array(bin_test['R_ame'])
redshift = np.array(bin_test['z'])
redshift_err = np.array(bin_test['z_err'])
richness_err = np.array(bin_test['R_ame_err'])

z_obs_params = Ncm.Matrix.new(len(redshift_err), 2)
lnM_obs_params = Ncm.Matrix.new(len(richness_err), 1)
for i in range(len(redshift_err)):
    lnM_obs_params.set(i , 0 ,richness_err[i] )
    for j in range(0, 2):
        if j == 0:
            z_obs_params.set(i, j, 0)
        if j == 1:
            z_obs_params.set(i, j,  redshift_err[i])
print(richness) 

ncdata.set_lnM_obs(Ncm.Matrix.new_array(richness, 1))
ncdata.set_z_obs(Ncm.Matrix.new_array(redshift, 1))
ncdata.set_z_obs_params(z_obs_params)
ncdata.set_lnM_obs_params(lnM_obs_params)
ncdata.true_data(False)

dset = Ncm.Dataset.new_array([ncdata])
likelihood = Ncm.Likelihood.new(dset)

experiment = Ncm.ObjDictStr()
experiment.set("likelihood", likelihood)
experiment.set("model-set", mset)
ser = Ncm.Serialize.new(Ncm.SerializeOpt.CLEAN_DUP)
ser.dict_str_to_yaml_file(experiment, "unbinned.yaml")

ca =[]
lnM_obs = ncdata.get_lnM_obs()
z_obs = ncdata.get_z_obs()
z_obs_params = ncdata.get_z_obs_params()



rich = np.linspace(15.76, 100 , 100)
redshift = np.linspace(0.0, 0.7 , 2)
rich_plot = []
cad.prepare(cosmo , cluster_z , cluster_m)
cad.set_area(area * (np.pi / 180) ** 2)
print(cad.n(cosmo , cluster_z , cluster_m))
halo = []
for i in range(len(rich)-1):
    z = []
    z_err = []
    r = []
    r_err = []
    bin_test =  catalog_richness_df.query(' R_ame >= %f   & R_ame <= %f  &  z >= %f  & z <= %f' % (rich[i] , rich[i+1] , 0 , 0.7))
    
    for j in range(len(bin_test['R_ame'])):
        r.append(bin_test['R_ame'].values[j] *bin_test['R_ame_err'].values[j])
        z.append(bin_test['z'].values[j] *bin_test['z_err'].values[j])
    
    mean_r = sum(r)/bin_test['R_ame_err'].sum()
    mean_z = sum(z)/bin_test['z_err'].sum()

    for j in range(len(bin_test['R_ame'])):
        r_err.append(bin_test['R_ame_err'].values[j] *(bin_test['R_ame'].values[j]- mean_r)**2)
        z_err.append(bin_test['z_err'].values[j]*(bin_test['z'].values[j] - mean_z)**2)
    
    sigma_r = np.sqrt(sum(r_err)/bin_test['R_ame_err'].sum())
    sigma_z = np.sqrt(sum(z_err)/bin_test['z_err'].sum())
    halo.append(cad.intp_bin_d2n(cosmo  , cluster_z , cluster_m  ,[rich[i]] , [rich[i+1]] , [sigma_r],[0.0] , [0.7] , [0 , sigma_z]))
    rich_plot.append((rich[i]+rich[i+1])/2)

In [ ]:
plt.hist(bin_test['R_ame'] , bins=100)
plt.plot(rich_plot , halo)
plt.xlim(0 , 100)
plt.yscale("log")

plt.show()